In [1]:
%load_ext autoreload
%autoreload 2
import torch
from wkv_kernel import WKV, WKVConfig
from einops import rearrange, reduce, repeat


/system/apps/userenv/beck/rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Translate the recurrence formula from CUDA to torch

In [2]:
# Setup cuda kernel
wkv_cuda_kernel = WKV()

Using /system/user/beck/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /system/user/beck/.cache/torch_extensions/py310_cu117/wkv/build.ninja...
Building extension module wkv...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module wkv...


### Inputs

In [3]:
dtype = torch.float32
device = torch.device('cuda')

In [4]:
# original input shapes from training
batch_size = 12
seq_len = 512
embedding_dim = 512
time_decay = torch.randn(
    (embedding_dim, ), dtype=dtype,
    device=device).to(memory_format=torch.contiguous_format)
time_first = torch.randn(
    (embedding_dim, ), dtype=dtype,
    device=device).to(memory_format=torch.contiguous_format)
k = torch.randn((batch_size, seq_len, embedding_dim),
                dtype=dtype,
                device=device).to(memory_format=torch.contiguous_format)
v = torch.randn((batch_size, seq_len, embedding_dim),
                device=device,
                dtype=dtype).to(memory_format=torch.contiguous_format)
y_gt = torch.empty(batch_size, seq_len, embedding_dim, dtype=dtype,
                device=device).to(memory_format=torch.contiguous_format)

In [5]:
# mock up for experiment
# batch_size = 2
# seq_len = 4
# embedding_dim = 2
# entries = batch_size * seq_len * embedding_dim
# time_decay = torch.arange(embedding_dim, dtype=dtype,
#                           device=device).reshape(embedding_dim).to(
#                               memory_format=torch.contiguous_format)
# time_first = torch.arange(embedding_dim, dtype=dtype,
#                           device=device).reshape(embedding_dim).to(
#                               memory_format=torch.contiguous_format)
# k = torch.arange(entries, dtype=dtype, device=device).reshape(
#     batch_size, seq_len,
#     embedding_dim).to(memory_format=torch.contiguous_format)
# v = torch.arange(entries, dtype=dtype, device=device).reshape(
#     batch_size, seq_len,
#     embedding_dim).to(memory_format=torch.contiguous_format)
# y_gt = torch.empty(batch_size, seq_len, embedding_dim, dtype=dtype,
#                 device=device).to(memory_format=torch.contiguous_format)


### CUDA version

In [6]:
wkv_cuda_kernel.wkv_cuda.forward(batch_size, seq_len, embedding_dim, time_decay,
                             time_first, k, v, y_gt)

In [7]:
y_gt

tensor([[[ 0.5230,  1.4994, -0.4554,  ...,  1.0435,  0.8593,  0.9639],
         [ 0.5368, -0.2667,  0.8389,  ...,  0.5313,  0.4112, -0.3319],
         [ 0.5379, -0.7265,  1.3164,  ...,  0.3360,  0.2319, -1.1678],
         ...,
         [ 0.5239, -0.0605, -0.2721,  ...,  0.3394,  0.2285, -0.3610],
         [ 0.5239,  0.7317, -0.3268,  ...,  0.3394,  0.2285, -0.3610],
         [ 0.5239,  0.6710, -0.0888,  ...,  0.3394,  0.2285, -0.3610]],

        [[-0.8473,  1.1180,  0.4535,  ...,  0.5746,  1.9634,  0.7242],
         [-0.7122,  0.0837, -0.2143,  ...,  0.2922,  1.6636, -0.2994],
         [-0.8010, -0.7374, -0.2310,  ...,  0.2071,  1.6797,  1.5611],
         ...,
         [-0.8333, -0.4574,  0.6207,  ...,  0.1914,  1.4570,  1.0155],
         [-0.8333, -0.3412, -0.4753,  ...,  0.1914,  1.4570,  1.0155],
         [-0.8333, -0.2502, -0.5761,  ...,  0.1914,  1.4570,  1.0155]],

        [[-0.9110,  0.3784,  0.4915,  ...,  0.3639, -0.5931,  0.5076],
         [-0.7329, -0.4737,  0.5772,  ...,  0

### Reproduced torch version 1

In [8]:
# ChatGPT output:

# ww = u_timefirst[c] + k[b, i, c]
# p = max(pp, ww)
# e1 = exp(pp - p)
# e2 = exp(ww - p)
# y[b, i, c] = (e1 * aa + e2 * v[b, i, c]) / (e1 * bb + e2)
# ww = w_timedecay[c] + pp
# p = max(ww, k[b, i, c])
# e1 = exp(ww - p)
# e2 = exp(k[b, i, c] - p)
# aa = e1 * aa + e2 * v[b, i, c]
# bb = e1 * bb + e2
# pp = p


In [9]:
def cuda_forward_mock(batch_size, seq_len, embedding_dim, time_decay,
                      time_first, k, v, y):
    y = torch.zeros(batch_size, seq_len, embedding_dim, dtype=dtype,
                device=device).to(memory_format=torch.contiguous_format)
    MIN_VAL = -1e-38
    for b in range(batch_size):
        for c in range(embedding_dim):
            pp = MIN_VAL
            aa = 0
            bb = 0
            for i in range(seq_len):
                # ii = i * embedding_dim + c
                kk = k[b, i, c]
                vv = v[b, i, c]
                ww = time_first[c] + kk
                p = torch.tensor(max(pp, ww))
                e1 = torch.exp(pp - p)
                e2 = torch.exp(ww - p)
                new_y = (e1 * aa + e2 * vv) / (e1 * bb + e2)
                y[b, i, c] = new_y
                ww = time_decay[c] + pp
                p = torch.tensor(max(ww, kk))
                e1 = torch.exp(ww - p)
                e2 = torch.exp(kk - p) # uses current key
                aa = e1 * aa + e2 * vv
                bb = e1 * bb + e2
                pp = p

    return y

In [10]:
# y_m = cuda_forward_mock(batch_size, seq_len, embedding_dim, time_decay,
#                         time_first, k, v, y)
# y_m

### torch version v3 (own impl)

In [11]:
def cuda_forward_mock3(batch_size, seq_len, embedding_dim, time_decay,
                      time_first, k, v, y):
    y = torch.zeros(batch_size, seq_len, embedding_dim, dtype=dtype,
                device=device).to(memory_format=torch.contiguous_format)
    MIN_VAL = -1e-38
    # reshape inputs
    k_ = rearrange(k, 'b s e -> s b e')
    v_ = rearrange(v, 'b s e -> s b e')
    y_ = rearrange(y, 'b s e -> s b e')
    tf = repeat(time_first, 'e -> b e', b=batch_size)
    td = repeat(time_decay, 'e -> b e', b=batch_size)
    # running sums
    aa = torch.zeros(batch_size, embedding_dim, dtype=dtype, device=device)
    bb = torch.zeros(batch_size, embedding_dim, dtype=dtype, device=device)
    pp = torch.full((batch_size, embedding_dim), MIN_VAL, dtype=dtype, device=device)
    for t in range(seq_len):
        ww = tf + k_[t]
        p = torch.max(pp, ww)
        e1 = torch.exp(pp - p)
        e2 = torch.exp(ww - p)
        y_[t] = (e1 * aa + e2 * v_[t]) / (e1 * bb + e2)
        ww = td + pp
        p = torch.max(ww, k_[t])
        e1 = torch.exp(ww - p)
        e2 = torch.exp(k_[t] - p)
        aa = e1 * aa + e2 * v_[t]
        bb = e1 * bb + e2
        pp = p
    y = rearrange(y_, 's b e -> b s e')
    return y

In [12]:
y_m3 = cuda_forward_mock3(batch_size, seq_len, embedding_dim, time_decay, time_first, k, v, y_gt)

In [13]:
y_m3

tensor([[[ 0.5230,  1.4994, -0.4554,  ...,  1.0435,  0.8593,  0.9639],
         [ 0.5368, -0.2667,  0.8389,  ...,  0.5313,  0.4112, -0.3319],
         [ 0.5379, -0.7265,  1.3164,  ...,  0.3360,  0.2319, -1.1678],
         ...,
         [ 0.5239, -0.0605, -0.2721,  ...,  0.3394,  0.2285, -0.3610],
         [ 0.5239,  0.7317, -0.3268,  ...,  0.3394,  0.2285, -0.3610],
         [ 0.5239,  0.6710, -0.0888,  ...,  0.3394,  0.2285, -0.3610]],

        [[-0.8473,  1.1180,  0.4535,  ...,  0.5746,  1.9634,  0.7242],
         [-0.7122,  0.0837, -0.2143,  ...,  0.2922,  1.6636, -0.2994],
         [-0.8010, -0.7374, -0.2310,  ...,  0.2071,  1.6797,  1.5611],
         ...,
         [-0.8333, -0.4574,  0.6207,  ...,  0.1914,  1.4570,  1.0155],
         [-0.8333, -0.3412, -0.4753,  ...,  0.1914,  1.4570,  1.0155],
         [-0.8333, -0.2502, -0.5761,  ...,  0.1914,  1.4570,  1.0155]],

        [[-0.9110,  0.3784,  0.4915,  ...,  0.3639, -0.5931,  0.5076],
         [-0.7329, -0.4737,  0.5772,  ...,  0

In [14]:
print(torch.allclose(y_gt, y_m3, atol=1e-5))

True


In [15]:
y_gt[:2, :2, :2], y_m3[:2, :2, :2]

(tensor([[[ 0.5230,  1.4994],
          [ 0.5368, -0.2667]],
 
         [[-0.8473,  1.1180],
          [-0.7122,  0.0837]]], device='cuda:0'),
 tensor([[[ 0.5230,  1.4994],
          [ 0.5368, -0.2667]],
 
         [[-0.8473,  1.1180],
          [-0.7122,  0.0837]]], device='cuda:0'))

#### Single computation steps

In [16]:
y = torch.zeros(batch_size, seq_len, embedding_dim, dtype=dtype,
            device=device).to(memory_format=torch.contiguous_format)

In [17]:
batch_size, seq_len, embedding_dim

(12, 512, 512)

In [18]:
k.shape, v.shape, y.shape, time_decay.shape, time_first.shape

(torch.Size([12, 512, 512]),
 torch.Size([12, 512, 512]),
 torch.Size([12, 512, 512]),
 torch.Size([512]),
 torch.Size([512]))

In [19]:
k_ = rearrange(k, 'b s e -> s b e')
v_ = rearrange(v, 'b s e -> s b e')
y_ = rearrange(y, 'b s e -> s b e')
tf = repeat(time_first, 'e -> b e', b=batch_size)
td = repeat(time_decay, 'e -> b e', b=batch_size)

In [20]:
k_.shape, v_.shape, y.shape, td.shape, tf.shape

(torch.Size([512, 12, 512]),
 torch.Size([512, 12, 512]),
 torch.Size([12, 512, 512]),
 torch.Size([12, 512]),
 torch.Size([12, 512]))

In [21]:
MIN_VAL = -1e-38
aa = torch.zeros(batch_size, embedding_dim, dtype=dtype, device=device)
bb = torch.zeros(batch_size, embedding_dim, dtype=dtype, device=device)
pp = torch.full((batch_size, embedding_dim), MIN_VAL, dtype=dtype, device=device)

In [22]:
t = 0

In [23]:
ww = tf + k_[t]


In [24]:
p = torch.max(pp, ww)

In [25]:
e1 = torch.exp(pp - p)
e2 = torch.exp(ww - p)

In [26]:
# y[t] = (e1 * aa + e2 * v_[t]) / (e1 * bb + e2)

In [27]:
ww = td + pp

In [28]:
ww

tensor([[ 2.2691, -2.6967, -1.3070,  ...,  1.1614,  0.4343,  0.4372],
        [ 2.2691, -2.6967, -1.3070,  ...,  1.1614,  0.4343,  0.4372],
        [ 2.2691, -2.6967, -1.3070,  ...,  1.1614,  0.4343,  0.4372],
        ...,
        [ 2.2691, -2.6967, -1.3070,  ...,  1.1614,  0.4343,  0.4372],
        [ 2.2691, -2.6967, -1.3070,  ...,  1.1614,  0.4343,  0.4372],
        [ 2.2691, -2.6967, -1.3070,  ...,  1.1614,  0.4343,  0.4372]],
       device='cuda:0')

In [29]:
p = torch.max(ww, k_[t])

In [30]:
e1 = torch.exp(ww - p)
e2 = torch.exp(k_[t] - p)

In [31]:
aa = e1 * aa + e2 * v_[t]
bb = e1 * bb + e2

In [32]:
pp = p

### torch version v4 (own impl, simplify)

In [33]:
def cuda_forward_mock4(batch_size, seq_len, embedding_dim, time_decay,
                      time_first, k, v, y):
    y = torch.zeros(batch_size, seq_len, embedding_dim, dtype=dtype,
                device=device).to(memory_format=torch.contiguous_format)
    MIN_VAL = -1e-38
    # reshape inputs
    k_ = rearrange(k, 'b s e -> s b e')
    v_ = rearrange(v, 'b s e -> s b e')
    y_ = rearrange(y, 'b s e -> s b e')
    tf = repeat(time_first, 'e -> b e', b=batch_size)
    td = repeat(time_decay, 'e -> b e', b=batch_size)
    # running sums
    aa = torch.zeros(batch_size, embedding_dim, dtype=dtype, device=device)
    bb = torch.zeros(batch_size, embedding_dim, dtype=dtype, device=device)
    pp = torch.full((batch_size, embedding_dim), MIN_VAL, dtype=dtype, device=device)
    for t in range(seq_len):
        # ww = tf + k_[t]
        # p = torch.max(pp, ww)
        # e1 = torch.exp(pp - p)
        # e2 = torch.exp(ww - p)
        # y_[t] = (e1 * aa + e2 * v_[t]) / (e1 * bb + e2)
        # ww = td + pp
        # p = torch.max(ww, k_[t])
        # e1 = torch.exp(ww - p)
        # e2 = torch.exp(k_[t] - p)
        # aa = e1 * aa + e2 * v_[t]
        # bb = e1 * bb + e2
        # pp = p
        ww = tf + k_[t]
        p = torch.max(pp, ww)
        e1 = torch.exp(pp - p)
        e2 = torch.exp(ww - p)
        y_[t] = (e1 * aa + e2 * v_[t]) / (e1 * bb + e2)
        ww = td + pp
        p = torch.max(ww, k_[t])
        e1 = torch.exp(ww - p)
        e2 = torch.exp(k_[t] - p)
        aa = e1 * aa + e2 * v_[t]
        bb = e1 * bb + e2
        pp = p
    y = rearrange(y_, 's b e -> b s e')
    return y

In [34]:
y_m4 = cuda_forward_mock4(batch_size, seq_len, embedding_dim, time_decay, time_first, k, v, y_gt)

In [35]:
y_m4

tensor([[[ 0.5230,  1.4994, -0.4554,  ...,  1.0435,  0.8593,  0.9639],
         [ 0.5368, -0.2667,  0.8389,  ...,  0.5313,  0.4112, -0.3319],
         [ 0.5379, -0.7265,  1.3164,  ...,  0.3360,  0.2319, -1.1678],
         ...,
         [ 0.5239, -0.0605, -0.2721,  ...,  0.3394,  0.2285, -0.3610],
         [ 0.5239,  0.7317, -0.3268,  ...,  0.3394,  0.2285, -0.3610],
         [ 0.5239,  0.6710, -0.0888,  ...,  0.3394,  0.2285, -0.3610]],

        [[-0.8473,  1.1180,  0.4535,  ...,  0.5746,  1.9634,  0.7242],
         [-0.7122,  0.0837, -0.2143,  ...,  0.2922,  1.6636, -0.2994],
         [-0.8010, -0.7374, -0.2310,  ...,  0.2071,  1.6797,  1.5611],
         ...,
         [-0.8333, -0.4574,  0.6207,  ...,  0.1914,  1.4570,  1.0155],
         [-0.8333, -0.3412, -0.4753,  ...,  0.1914,  1.4570,  1.0155],
         [-0.8333, -0.2502, -0.5761,  ...,  0.1914,  1.4570,  1.0155]],

        [[-0.9110,  0.3784,  0.4915,  ...,  0.3639, -0.5931,  0.5076],
         [-0.7329, -0.4737,  0.5772,  ...,  0

In [36]:
print(torch.allclose(y_gt, y_m4, atol=1e-5))

True
